CS4001/4042 Assignment 1, Part B, Q4
---

Model degradation is a common issue faced when deploying machine learning models (including neural networks) in the real world. New data points could exhibit a different pattern from older data points due to factors such as changes in government policy or market sentiments. For instance, housing prices in Singapore have been increasing and the Singapore government has introduced 3 rounds of cooling measures over the past years (16 December 2021, 30 September 2022, 27 April 2023).

In such situations, the distribution of the new data points could differ from the original data distribution which the models were trained on. Recall that machine learning models often work with the assumption that the test distribution should be similar to train distribution. When this assumption is violated, model performance will be adversely impacted.  In the last part of this assignment, we will investigate to what extent model degradation has occurred.




---



---



Your co-investigators used a linear regression model to rapidly test out several combinations of train/test splits and shared with you their findings in a brief report attached in Appendix A below. You wish to investigate whether your deep learning model corroborates with their findings.

In [ ]:
!pip install alibi-detect -q

In [ ]:
SEED = 42

import os

import random
random.seed(SEED)

import numpy as np
np.random.seed(SEED)

import pandas as pd

from alibi_detect.cd import TabularDrift

> Evaluate your model from B1 on data from year 2022 and report the test R2.

In [ ]:
import pytorch_tabular
from pytorch_tabular import TabularModel
from pytorch_tabular.models import CategoryEmbeddingModelConfig
from pytorch_tabular.config import (
    DataConfig,
    OptimizerConfig,
    TrainerConfig,
)
from sklearn.metrics import r2_score, mean_squared_error

df = pd.read_csv('hdb_price_prediction.csv')

# TODO: Enter your code here

mask_train = (df['year']<=2019)
mask_val = (df['year']==2020)
mask_test_2022 = (df['year']==2022)
mask_test_2023 = (df['year']==2023)

train = df.loc[mask_train]
val = df.loc[mask_val]
test_2022 = df.loc[mask_test_2022]
test_2023 = df.loc[mask_test_2023]

target = 'resale_price'
num_col_names = ['dist_to_nearest_stn', 'dist_to_dhoby', 'degree_centrality', 'eigenvector_centrality', 'remaining_lease_years', 'floor_area_sqm']
cat_col_names = ['month', 'town', 'flat_model_type', 'storey_range']

## Using model from B1
tabular_model = TabularModel.load_from_checkpoint("models/partb1")
pred_df_2022 = tabular_model.predict(test_2022)

y = pred_df_2022['resale_price']
y_pred = pred_df_2022['resale_price_prediction']

r2 = r2_score(y, y_pred)

print(f"R2: {r2}")

2023-10-06 19:28:03,507 - {pytorch_tabular.tabular_model:129} - INFO - Experiment Tracking is turned off
INFO:pytorch_tabular.tabular_model:Experiment Tracking is turned off
2023-10-06 19:28:03,532 - {pytorch_tabular.tabular_model:268} - INFO - Preparing the Trainer
INFO:pytorch_tabular.tabular_model:Preparing the Trainer
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:589: LightningDeprecationWarning: The Trainer argument `auto_select_gpus` has been deprecated in v1.9.0 and will be removed in v2.0.0. Please use the function `pytorch_lightning.accelerators.find_usable_cuda_devices` instead.
  rank_zero_deprecation(
INFO:pytorch_lightning.utilities.rank_zero:Trainer already configured with model summary callbacks: [<class 'pytorch_lightning.callbacks.rich_model_summary.RichModelSummary'>]. Skipping setting a default `ModelSummary` callback.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_ligh

Output()

R2: 0.43884732217645395


> Evaluate your model from B1 on data from year 2023 and report the test R2.

In [ ]:
# TODO: Enter your code here

## Using model from B1
pred_df_2023 = tabular_model.predict(test_2023)

y = pred_df_2023['resale_price']
y_pred = pred_df_2023['resale_price_prediction']

r2 = r2_score(y, y_pred)

print(f"R2: {r2}")

Output()

R2: 0.16212595367206906


> Did model degradation occur for the deep learning model?


\<
Model degradation did occur for the model trained on data from 2019 and before. The test R2 decreased in year 2022 to 0.438 and further decreased in year 2023 to 0.162, indicating that the model is no longer able to generalise well and that the learned weights fromm 2019 data is outdated, and no longer applies to recent data due to a shift in distribution of the data.\>



---

---



Model degradation could be caused by [various data distribution shifts](https://huyenchip.com/2022/02/07/data-distribution-shifts-and-monitoring.html#data-shift-types): covariate shift (features), label shift and/or concept drift (altered relationship between features and labels).
There are various conflicting terminologies in the [literature](https://www.sciencedirect.com/science/article/pii/S0950705122002854#tbl1). Let’s stick to this reference for this assignment.

> Using the **Alibi Detect** library, apply the **TabularDrift** function with the training data (year 2019 and before) used as the reference and **detect which features have drifted** in the 2023 test dataset. Before running the statistical tests, ensure you **sample 1000 data points** each from the train and test data. Do not use the whole train/test data. (Hint: use this example as a guide https://docs.seldon.io/projects/alibi-detect/en/stable/examples/cd_chi2ks_adult.html)


In [ ]:
# TODO: Enter your code here
target = 'resale_price'
num_col_names = ['dist_to_nearest_stn', 'dist_to_dhoby', 'degree_centrality', 'eigenvector_centrality', 'remaining_lease_years', 'floor_area_sqm']
cat_col_names = ['month', 'town', 'flat_model_type', 'storey_range']

feature_names = num_col_names+cat_col_names
category_map = {}

## from https://docs.seldon.io/projects/alibi/en/latest/_modules/alibi/datasets/default.html#fetch_adult, category map has integers corresponding to index of categorical columns as keys and unique classes as values
for f in cat_col_names:
    category_map[feature_names.index(f)] = list(train[f].unique())

n_samples = 1000

train = train[:n_samples]
test_2023 = test_2023[:n_samples]

X_ref = train[feature_names].to_numpy()

X_test = test_2023[feature_names].to_numpy()

categories_per_feature = {f: None for f in list(category_map.keys())}
cd = TabularDrift(X_ref, p_val=.05, categories_per_feature=categories_per_feature)

print("\n========================= OVERALL DRIFT =========================")
preds = cd.predict(X_test)
labels = ['No!', 'Yes!']
print('Drift? {}'.format(labels[preds['data']['is_drift']]))

print("\n========================= FEATURE CONTRIBTION TO OVERALL =========================")
for f in range(cd.n_features):
    stat = 'Chi2' if f in list(categories_per_feature.keys()) else 'K-S'
    fname = feature_names[f]
    stat_val, p_val = preds['data']['distance'][f], preds['data']['p_val'][f]
    print(f'{fname} -- {stat} {stat_val:.3f} -- p-value {p_val:.3f}')

print("\n========================= INDIVIDUAL FEATURE DRIFT =========================")
fpreds = cd.predict(X_test, drift_type='feature')
for f in range(cd.n_features):
    stat = 'Chi2' if f in list(categories_per_feature.keys()) else 'K-S'
    fname = feature_names[f]
    is_drift = fpreds['data']['is_drift'][f]
    stat_val, p_val = fpreds['data']['distance'][f], fpreds['data']['p_val'][f]
    print(f'{fname} -- Drift? {labels[is_drift]} -- {stat} {stat_val:.3f} -- p-value {p_val:.3f}')


========================= OVERALL DRIFT =========================
Drift? Yes!

========================= FEATURE CONTRIBTION TO OVERALL =========================
dist_to_nearest_stn -- K-S 0.055 -- p-value 0.094
dist_to_dhoby -- K-S 0.218 -- p-value 0.000
degree_centrality -- K-S 0.029 -- p-value 0.783
eigenvector_centrality -- K-S 0.195 -- p-value 0.000
remaining_lease_years -- K-S 0.271 -- p-value 0.000
floor_area_sqm -- K-S 0.134 -- p-value 0.000
month -- Chi2 0.000 -- p-value 1.000
town -- Chi2 667.474 -- p-value 0.000
flat_model_type -- Chi2 77.586 -- p-value 0.000
storey_range -- Chi2 38.800 -- p-value 0.001

========================= INDIVIDUAL FEATURE DRIFT =========================
dist_to_nearest_stn -- Drift? No! -- K-S 0.055 -- p-value 0.094
dist_to_dhoby -- Drift? Yes! -- K-S 0.218 -- p-value 0.000
degree_centrality -- Drift? No! -- K-S 0.029 -- p-value 0.783
eigenvector_centrality -- Drift? Yes! -- K-S 0.195 -- p-value 0.000
remaining_lease_years -- Drift? Yes! -- K-S 0.

> Assuming that the flurry of housing measures have made an impact on the relationship between all the features and resale_price (i.e. P(Y|X) changes), which type of data distribution shift possibly led to model degradation?


\<Considering the above assumption of altered relationship between features and labels (resale price), the drift is most likely due to a concept drift\>

> From your analysis via TabularDrift, which features contribute to this shift?


\<TabularDrift analysis above indicates that the responsible features are
- dist_to_dhoby
- eigenvector_centrality
- remaining_lease_years
- floor_area_sqm
- town
- flat_model_type
- storey_range\>

> Suggest 1 way to address model degradation and implement it, showing improved test R2 for year 2023.


\<To improve model degradation, we can use more recent data for training. Instead of using only data from 2019 and before,
we can use data from 2021-2022 (when cooling measures happened) and test on 2023 data. This resulted in a much better R2 score of 0.764\>

In [5]:
# TODO: Enter your code here

new_mask_train = (df['year']<=2022) & (df['year']>=2021)
new_train = df.loc[new_mask_train]
tabular_model.fit(train = new_train)

result = tabular_model.evaluate(test_2023)
pred_df = tabular_model.predict(test_2023)

y = pred_df['resale_price']
y_pred = pred_df['resale_price_prediction']
r2 = r2_score(y, y_pred)

print("\n========================= RETRAINED MODEL SCORE =========================")
print(f"R2: {r2}")

INFO:lightning_fabric.utilities.seed:Global seed set to 42
2023-10-06 20:39:03,231 - {pytorch_tabular.tabular_model:473} - INFO - Preparing the DataLoaders
INFO:pytorch_tabular.tabular_model:Preparing the DataLoaders
2023-10-06 20:39:03,244 - {pytorch_tabular.tabular_datamodule:290} - INFO - Setting up the datamodule for regression task
INFO:pytorch_tabular.tabular_datamodule:Setting up the datamodule for regression task
2023-10-06 20:39:03,534 - {pytorch_tabular.tabular_model:521} - INFO - Preparing the Model: CategoryEmbeddingModel
INFO:pytorch_tabular.tabular_model:Preparing the Model: CategoryEmbeddingModel
2023-10-06 20:39:03,616 - {pytorch_tabular.tabular_model:268} - INFO - Preparing the Trainer
INFO:pytorch_tabular.tabular_model:Preparing the Trainer
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:589: LightningDeprecationWarning: The Trainer argument `auto_select_gpus` has been deprecated in v1.9.0 and will be removed in v2

Finding best initial lr:   0%|          | 0/100 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/core/module.py:493: UserWarning: You called `self.log('train_loss', ..., logger=True)` but have no logger configured. You can enable one by doing `Trainer(logger=ALogger(...))`
  rank_zero_warn(
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/core/module.py:493: UserWarning: You called `self.log('train_mean_squared_error', ..., logger=True)` but have no logger configured. You can enable one by doing `Trainer(logger=ALogger(...))`
  rank_zero_warn(
INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_steps=100` reached.
INFO:pytorch_lightning.tuner.lr_finder:Learning rate set to 0.6918309709189363
INFO:pytorch_lightning.utilities.rank_zero:Restoring states from the checkpoint path at /content/content/MyDrive/cz4042 neural networks/nn_assignment/.lr_find_1ba8f87a-e0ef-4674-8297-018644f55321.ckpt
INFO:pytorch_lightning.utilities.rank_zero:Restored all states from the checkpoint file at /content/content/M

┏━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃   ┃ Name             ┃ Type                      ┃ Params ┃
┡━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│ 0 │ _backbone        │ CategoryEmbeddingBackbone │  3.0 K │
│ 1 │ _embedding_layer │ Embedding1dLayer          │  1.6 K │
│ 2 │ head             │ LinearHead                │     51 │
│ 3 │ loss             │ MSELoss                   │      0 │
└───┴──────────────────┴───────────────────────────┴────────┘

Trainable params: 4.6 K                                                                                            
Non-trainable params: 0                                                                                            
Total params: 4.6 K                                                                                                
Total estimated model params size (MB): 0

...

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/core/module.py:493: UserWarning: You called 
`self.log('test_loss', ..., logger=True)` but have no logger configured. You can enable one by doing 
`Trainer(logger=ALogger(...))`
  rank_zero_warn(

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/core/module.py:493: UserWarning: You called 
`self.log('test_mean_squared_error', ..., logger=True)` but have no logger configured. You can enable one by doing 
`Trainer(logger=ALogger(...))`
  rank_zero_warn(

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │       6947910144.0        │
│  test_mean_squared_error  │       6947910144.0        │
└───────────────────────────┴───────────────────────────┘

Output()


========================= RETRAINED MODEL SCORE =========================
R2: 0.7643258295911679


### Appendix A



Here are our results from a linear regression model. We used StandardScaler for continuous variables and OneHotEncoder for categorical variables.

While 2021 data can be predicted well, test R2 dropped rapidly for 2022 and 2023.

| Training set | Test set | Test R2 |
|--------------|----------|---------|
| Year <= 2020 | 2021     | 0.76    |
| Year <= 2020 | **2022**     | 0.41    |
| Year <= 2020 | **2023**     | **0.10**   |



Similarly, a model trained on 2017 data can predict 2018-2021 well (with slight degradation in performance for 2021), but drops drastically in 2022 and 2023.

| Training set | Test set | Test R2 |
|--------------|----------|---------|
| 2017         | 2018     | 0.90    |
|              | 2019     | 0.89    |
|              | 2020     | 0.87    |
|              | 2021     | 0.72    |
|              | **2022**     | **0.37**    |
|              | **2023**     | **0.09**    |

With the test set fixed at year 2021, training on data from 2017-2020 still works well on the test data, with minimal degradation. Training sets closer to year 2021 generally do better.

| Training set | Test set | Test R2 |
|--------------|----------|---------|
| 2020         | 2021     | 0.81    |
| 2019         | 2021     | 0.75    |
| 2018         | 2021     | 0.73    |
| 2017         | 2021     | 0.72    |